In [27]:
# !pip install flask

     |████████████████████████████████| 94 kB 927 kB/s eta 0:00:011
     |████████████████████████████████| 82 kB 1.6 MB/s eta 0:00:011


In [2]:
import os
import json
import requests
from flask import Flask, Response

from dotenv import load_dotenv
load_dotenv()

True

In [8]:
app = Flask(__name__)

API_KEY = os.getenv('API_KEY')

In [9]:
def requestCurrentInfo(startIdx, endIdx):
    url = "http://openapi.seoul.go.kr:8088/{}/json/bikeList/{}/{}".format(API_KEY, startIdx, endIdx)
    res = requests.get(url)
    res = res.json()
    
    if res['rentBikeStatus']['RESULT']['CODE'] != 'INFO-000':
        raise Exception('Failed to get data from api.')
    
    return res['rentBikeStatus']['row']


def getAllCombinedInfo():
    res = []
    for i in range(1, 1001 + 1, 1000):
        startIdx = i
        endIdx = i + 999
        res = res + requestCurrentInfo(startIdx, endIdx)
        
    return res

In [10]:
MODELS_PATH = "./models/"

@app.route("/stations/available")
def get_stations():
    
    def isAvailableStation(station):
        if station['stationId'] in availableStations:
            return station
        else:
            return None
        
    file_list = os.listdir(MODELS_PATH)
    availableStations = list(map(lambda x: x.split('.')[0], file_list))
    
    allStations = getAllCombinedInfo()
        
    result = list(filter(isAvailableStation, allStations))
    resultInJson = json.dumps(result, ensure_ascii=False)
    return Response(resultInJson, mimetype='application/json')


In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
1.231.99.37 - - [30/Jul/2020 05:01:42] "GET /stations/available HTTP/1.1" 200 -
